In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Load Network 15AB

In [3]:
# model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #model
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_Full/models/B5R2_15AB_Full_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Testset1312.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
dataframe = pd.read_csv (r'/home/yupaporn/CCA_Project/UICCA_DiagRadioExp_AzureDb.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
print(dataframe .shape)
# dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# print(dataframe.shape)
# print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
# print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(813, 13)


,Unnamed: 0,Unnamed: 0.1,P ID,Site ID,date,originalImage,Path Full,FileName,Class,Sub_class_name,Sub_class,Path Crop,Path Crop_I7
0,0,0,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_1.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...
1,1,1,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_2.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...
2,2,2,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_3.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...
3,3,3,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_4.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...
4,4,4,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_5.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...


In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class',# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 813 validated image filenames belonging to 10 classes.
{0: "['AB01']", 1: "['AB02', 'AB11']", 2: "['AB02']", 3: "['AB03']", 4: "['AB05']", 5: "['AB06', 'AB081']", 6: "['AB09']", 7: "['AB10']", 8: "['AB11']", 9: "['Normal']"}


In [7]:
labels = {0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}

In [8]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [9]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.16216983,  0.37311855, -0.20677656, ...,  0.23740651,
        -0.13335361,  0.25989625],
       [ 0.20093489, -0.10753673, -0.17692952, ...,  0.01061908,
        -0.1985593 , -0.01361911],
       [ 0.4260717 ,  0.07038802, -0.10210872, ..., -0.0676544 ,
        -0.15870324,  0.12080023],
       ...,
       [-0.01751553, -0.0338412 , -0.15124118, ...,  0.09293045,
        -0.10729779,  0.37779596],
       [-0.12906888, -0.15738532,  0.07456718, ...,  0.02933789,
        -0.15200962, -0.13556109],
       [-0.1329987 ,  0.16837023, -0.2028814 , ...,  0.16930175,
        -0.01470976, -0.12365816]])

In [10]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.162170,0.373119,-0.206777,-0.090784,-0.106962,0.170049,-0.154019,-0.031511,-0.210201,-0.177617,...,0.116691,-0.221105,-0.089018,0.329051,-0.081738,-0.033136,-0.077009,0.237407,-0.133354,0.259896
1,0.200935,-0.107537,-0.176930,-0.183966,-0.095405,0.093535,-0.144208,-0.010653,-0.111979,-0.000766,...,0.107824,-0.159964,-0.028037,0.130123,-0.132022,-0.184636,0.061931,0.010619,-0.198559,-0.013619
2,0.426072,0.070388,-0.102109,-0.159541,-0.189294,0.219006,-0.098589,0.136929,-0.068871,-0.176865,...,0.217638,-0.193831,0.027975,0.149621,-0.142312,-0.042885,0.265620,-0.067654,-0.158703,0.120800
3,0.033356,-0.066455,-0.004623,-0.036767,-0.215879,0.243723,-0.112175,-0.170005,-0.021037,-0.088087,...,-0.022632,-0.150470,-0.143163,-0.064558,-0.177722,-0.119786,-0.130780,-0.138916,-0.184719,0.297594
4,-0.057476,0.013760,-0.219840,-0.147638,-0.093520,0.209551,-0.175600,-0.201921,-0.188871,-0.065324,...,-0.044642,-0.178038,-0.222533,0.492777,0.051294,-0.102051,-0.168922,0.332214,-0.190183,0.189209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,-0.036016,-0.030781,-0.002133,-0.115271,-0.156255,0.060351,-0.150695,-0.181863,-0.115627,-0.099838,...,0.026512,-0.098907,-0.107871,0.185931,-0.070227,-0.189797,-0.183593,0.122654,-0.041150,-0.112148
809,0.282880,0.054666,-0.187765,-0.020248,0.111334,0.130226,-0.186284,0.037036,0.307345,0.422600,...,0.265137,-0.142368,0.180570,0.698772,0.206022,-0.062219,-0.073437,0.294655,-0.099968,0.374662
810,-0.017516,-0.033841,-0.151241,0.254034,0.219540,-0.003998,-0.181064,-0.068055,0.229465,0.073023,...,0.392637,-0.197532,-0.135415,0.667018,0.294627,0.215776,-0.102109,0.092930,-0.107298,0.377796
811,-0.129069,-0.157385,0.074567,0.005774,-0.141309,0.228510,-0.196027,-0.195661,0.156221,0.307338,...,-0.045584,-0.064520,-0.158708,0.924970,-0.019510,-0.113676,-0.154120,0.029338,-0.152010,-0.135561


In [11]:
dataframe

,Unnamed: 0,Unnamed: 0.1,P ID,Site ID,date,originalImage,Path Full,FileName,Class,Sub_class_name,Sub_class,Path Crop,Path Crop_I7,category,Prob
0,0,0,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_1.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,0.999286
1,1,1,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_2.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,0.993507
2,2,2,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_3.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,0.985898
3,3,3,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_4.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,0.988357
4,4,4,12150,13777,11/10/2021,1.2.392.200039.110.1.109018720.10.20211011.134...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12150_5.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,0.999318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,808,808,12285,13777,27/10/2021,1.2.392.200039.110.1.109018960.10.20211027.151...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12285_11.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,AB12,0.999449
809,809,809,12285,13777,27/10/2021,1.2.392.200039.110.1.109018960.10.20211027.151...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12285_12.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,1.000000
810,810,810,12285,13777,27/10/2021,1.2.392.200039.110.1.109018960.10.20211027.151...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12285_13.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,1.000000
811,811,811,12285,13777,27/10/2021,1.2.392.200039.110.1.109018960.10.20211027.151...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,12285_14.jpg,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,AB12,0.761010


In [12]:
b = dataframe['P ID'].tolist()
b2 = dataframe['date'].tolist()
b3 = dataframe['Path Crop'].tolist()
b4 = dataframe['Class'].tolist()
b5 = dataframe['Sub_class_name'].tolist()
b6 = dataframe['Sub_class'].tolist()
b7 = dataframe['category'].tolist()
b8 = dataframe['Prob'].tolist()

In [13]:
dataset['P ID'] = b
dataset['date'] = b2
dataset['Path crop'] = b3
dataset['Class'] = b4
dataset['Sub_class_name'] = b5
dataset['Sub_class'] = b6
dataset['category'] = b7
dataset['Prob'] = b8

In [14]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2046,2047,P ID,date,Path crop,Class,Sub_class_name,Sub_class,category,Prob
0,0.162170,0.373119,-0.206777,-0.090784,-0.106962,0.170049,-0.154019,-0.031511,-0.210201,-0.177617,...,-0.133354,0.259896,12150,11/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,0.999286
1,0.200935,-0.107537,-0.176930,-0.183966,-0.095405,0.093535,-0.144208,-0.010653,-0.111979,-0.000766,...,-0.198559,-0.013619,12150,11/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,0.993507
2,0.426072,0.070388,-0.102109,-0.159541,-0.189294,0.219006,-0.098589,0.136929,-0.068871,-0.176865,...,-0.158703,0.120800,12150,11/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,0.985898
3,0.033356,-0.066455,-0.004623,-0.036767,-0.215879,0.243723,-0.112175,-0.170005,-0.021037,-0.088087,...,-0.184719,0.297594,12150,11/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,0.988357
4,-0.057476,0.013760,-0.219840,-0.147638,-0.093520,0.209551,-0.175600,-0.201921,-0.188871,-0.065324,...,-0.190183,0.189209,12150,11/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,0.999318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,-0.036016,-0.030781,-0.002133,-0.115271,-0.156255,0.060351,-0.150695,-0.181863,-0.115627,-0.099838,...,-0.041150,-0.112148,12285,27/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],AB12,0.999449
809,0.282880,0.054666,-0.187765,-0.020248,0.111334,0.130226,-0.186284,0.037036,0.307345,0.422600,...,-0.099968,0.374662,12285,27/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,1.000000
810,-0.017516,-0.033841,-0.151241,0.254034,0.219540,-0.003998,-0.181064,-0.068055,0.229465,0.073023,...,-0.107298,0.377796,12285,27/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],Normal,1.000000
811,-0.129069,-0.157385,0.074567,0.005774,-0.141309,0.228510,-0.196027,-0.195661,0.156221,0.307338,...,-0.152010,-0.135561,12285,27/10/2021,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,['Normal'],['Normal'],AB12,0.761010


In [15]:
set(dataset[dataset['Class'] == 'Normal']['Sub_class_name'])

{"['Normal']"}

In [16]:
os.chdir( '/home/yupaporn/CCA_Project')
testdf = dataset
testdf.to_csv('FeatureVector_CCA_case_DiagRadioExp_AzureDb813_modelFull.csv') 